# CNM_CycNucMed — Clean Colab/Jupyter Runner (run_pipeline)

This notebook is minimal and avoids argparse issues by calling the **library** entrypoint:

```python
from iaea_project.pipeline import run_pipeline
```

If you edited `pipeline.py`, re-run the install cell (or restart the runtime) so Colab picks up changes.


In [ ]:
# =========================
# 1) Clone / pull + cd
# =========================
import os, subprocess, pathlib

REPO_URL = "https://github.com/tommasocarzaniga/CNM_CycNucMed.git"
REPO_DIR = "CNM_CycNucMed"

def sh(cmd, cwd=None):
    print(">>", cmd)
    subprocess.check_call(cmd, shell=True, cwd=cwd)

# If you get an auth error, your repo is private.
# Clone using a GitHub Personal Access Token (PAT):
#   from getpass import getpass
#   token = getpass("GitHub PAT (will not be shown): ").strip()
#   sh(f"git clone https://{token}@github.com/tommasocarzaniga/CNM_CycNucMed.git {REPO_DIR}")

if not os.path.exists(REPO_DIR):
    sh(f"git clone {REPO_URL}")
else:
    sh("git pull", cwd=REPO_DIR)

%cd {REPO_DIR}
print("Now in:", pathlib.Path().resolve())


In [ ]:
# =========================
# 2) Install deps + package (editable)
# =========================
import sys, subprocess, pathlib

def pip(cmd: str):
    print(">> pip", cmd)
    subprocess.check_call([sys.executable, "-m", "pip"] + cmd.split())

pip("install -U pip setuptools wheel")

root = pathlib.Path('.').resolve()
if (root / 'requirements.txt').exists():
    pip(f"install -r {root/'requirements.txt'}")

# Editable install (pyproject.toml / src layout supported)
pip("install -e .")


In [ ]:
# =========================
# 3) Verify pipeline entrypoint (avoid 'main'/argparse)
# =========================
import importlib
import iaea_project.pipeline as pipeline

# Force reload in case you edited files after the first import
pipeline = importlib.reload(pipeline)

print("pipeline module:", pipeline.__file__)
print("Has run_pipeline:", hasattr(pipeline, "run_pipeline"))
assert hasattr(pipeline, "run_pipeline"), (
    "run_pipeline() not found. Make sure src/iaea_project/pipeline.py defines run_pipeline and that you re-ran the install cell."
)


In [ ]:
# =========================
# 4) Run via library API
# =========================
from iaea_project.pipeline import run_pipeline

# Run all countries:
# pdf_path = run_pipeline()

# Run selected countries:
pdf_path = run_pipeline(["Switzerland"])  # edit as needed

print("Generated PDF at:", pdf_path)


In [ ]:
# =========================
# 5) Optional: Run via CLI wrapper
# =========================
# !python scripts/cyclotrons_run.py --countries Switzerland Germany
